In [11]:
from google.colab import files
uploaded =files.upload()

Saving 50_Startups (1).csv to 50_Startups (1) (1).csv


In [13]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
import matplotlib.pyplot as plt

data = pd.read_csv("50_Startups (1).csv")


print("Missing values:\n", data.isnull().sum())


label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


scaler = StandardScaler()
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])


def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

for col in numeric_columns:
    data = remove_outliers_iqr(data, col)


X = data.iloc[:, :-1]
y = data.iloc[:, -1]
f_values, p_values = f_classif(X, y)

fisher_scores = np.abs(f_values)

feature_scores = pd.DataFrame({"Feature": X.columns, "ANOVA_F": f_values, "Fisher_Score": fisher_scores})
feature_scores.sort_values(by="Fisher_Score", ascending=False, inplace=True)
print("Feature Selection Scores:\n", feature_scores)


print("Target variable unique values:\n", y.unique())
print("Target variable type:", y.dtype)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


if y.dtype in ['int64', 'float64'] and len(y.unique()) <= 10:

    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)


    print("Accuracy Score:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

elif y.dtype in ['int64', 'float64']:

    reg = LinearRegression()
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)


    print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
    print("Predictions:\n", y_pred)
else:
    print("The target variable is not valid for classification or regression.")


Missing values:
 R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64
Feature Selection Scores:
            Feature  ANOVA_F  Fisher_Score
0        R&D Spend      NaN           NaN
1   Administration      NaN           NaN
2  Marketing Spend      NaN           NaN
3            State      NaN           NaN
Target variable unique values:
 [ 2.01120333  1.99942997  1.98084225  1.77662724  1.35774012  1.12724963
  1.10548055  1.09620987  1.00746967  0.94602247  0.85484675  0.80816756
  0.74115484  0.55874952  0.51602637  0.44871967  0.3754357   0.33477114
  0.307116    0.26977265  0.16193522 -0.01753384 -0.04161264 -0.08216943
 -0.08672946 -0.11549309 -0.15736664 -0.17554233 -0.21879755 -0.27588222
 -0.3026246  -0.36412744 -0.36552389 -0.38178711 -0.38344421 -0.38929092
 -0.53393161 -0.5529549  -0.77149734 -0.77709368 -0.84641135 -0.85746568
 -1.01536466 -1.05896021 -1.17320899 -1.18008224 -1.56692212 -1.74062718
 -1.91321197

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:109: RuntimeWarning: invalid value encountered in divide
  msw = sswn / float(dfwn)
